In [2]:
import numpy as np
from stabalizer_circuit import StabilizerCircuit

# Circuit Functions

Define two helper functions: one to build a 2-qubit Bell pair, and one to build an n-qubit GHZ state.


In [13]:
# Define helper functions
def make_bell() -> StabilizerCircuit:
    sc = StabilizerCircuit(2)
    sc.h(0)
    sc.cx(0, 1)
    return sc

def make_ghz(n: int = 3) -> StabilizerCircuit:
    sc = StabilizerCircuit(n)
    sc.h(0)
    for i in range(1, n):
        sc.cx(0, i)
    return sc

# for printing
def format_complex(arr, tol=1e-8):
    """
    Format a complex scalar or numpy array with 4 decimal places,
    dropping negligible real or imaginary parts.

    Args:
      arr: A complex number or array-like of complex numbers.
      tol: Threshold below which a real/imag part is considered zero.

    Returns:
      If arr is scalar: a formatted string.
      If arr is array-like: prints each row with formatted entries.
    """
    def _fmt(z):
        if abs(z.imag) < tol:
            return f"{z.real:.4f}"
        if abs(z.real) < tol:
            return f"{z.imag:.4f}j"
        return f"{z.real:.4f}{z.imag:+.4f}j"

    arr_np = np.array(arr)
    if arr_np.ndim == 0:  # scalar
        return _fmt(arr_np.item())
    elif arr_np.ndim == 1:
        print("[", "  ".join(_fmt(z) for z in arr_np), "]")
    else:
        for row in arr_np:
            print("[", "  ".join(_fmt(z) for z in row), "]")


# Bell-Pair Analysis

1. Build and display the Bell-pair circuit.  
2. Compute and print the exact statevector.  
3. Perform full Pauli tomography on qubits [0,1] and print the reconstructed density matrix.  
4. Compare to the ideal density matrix and compute the tomography fidelity.


In [14]:
# Bell-pair example

# build the circuit
sc = make_bell()

# print the Stim circuit directly
print("Circuit:")
print(sc.circuit)

# exact statevector
psi = sc.state_vector()
print("\nStatevector:")
format_complex(psi)         

# tomography
rho = sc.tomography_dm([0, 1], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)      

# ideal density and fidelity
rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1

Statevector:
[ 0.7071  0.0000  0.0000  0.7071 ]

Reconstructed ρ:
[ 0.5072  -0.0098+0.0013j  -0.0003-0.0005j  0.5000-0.0063j ]
[ -0.0098-0.0013j  -0.0128  -0.0077j  0.0158-0.0100j ]
[ -0.0003+0.0005j  0.0077j  0.0127  0.0078-0.0027j ]
[ 0.5000+0.0063j  0.0158+0.0100j  0.0078+0.0027j  0.4928 ]

Ideal ρ:
[ 0.5000  0.0000  0.0000  0.5000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.0000  0.0000  0.0000  0.0000 ]
[ 0.5000  0.0000  0.0000  0.5000 ]

Fidelity: 1.0000


# 3-Qubit GHZ Analysis

1. Build and display the 3-qubit GHZ circuit.  
2. Compute and print the exact statevector.  
3. Perform full Pauli tomography on all three qubits and print the reconstructed density matrix.  
4. Compare to the ideal density matrix and compute the tomography fidelity.


In [15]:
# 3-qubit GHZ example

sc = make_ghz(3)

print("Circuit:")
print(sc.circuit)

psi = sc.state_vector()
print("\nStatevector:")
format_complex(psi)

rho = sc.tomography_dm([0, 1, 2], shots=2000)
print("\nReconstructed rho:")
format_complex(rho)

rho_ideal = np.outer(psi, psi.conj())
print("\nIdeal rho:")
format_complex(rho_ideal)

fidelity = np.real(psi.conj() @ (rho @ psi))
print("\nFidelity:", f"{fidelity:.4f}")


Circuit:
H 0
CX 0 1 0 2

Statevector:
[ 0.7071  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.7071 ]

Reconstructed ρ:
[ 0.4931  -0.0101-0.0033j  -0.0041+0.0019j  -0.0074+0.0039j  0.0029+0.0011j  -0.0061+0.0090j  -0.0096-0.0057j  0.5000+0.0065j ]
[ -0.0101+0.0033j  0.0014  0.0006+0.0069j  0.0061+0.0061j  0.0056-0.0017j  -0.0006+0.0091j  0.0040j  0.0026-0.0005j ]
[ -0.0041-0.0019j  0.0006-0.0069j  0.0021  -0.0039+0.0018j  -0.0076+0.0030j  -0.0007j  0.0009+0.0074j  -0.0086-0.0100j ]
[ -0.0074-0.0039j  0.0061-0.0061j  -0.0039-0.0018j  0.0099  0.0068j  0.0056-0.0033j  0.0001+0.0028j  -0.0001+0.0074j ]
[ 0.0029-0.0011j  0.0056+0.0017j  -0.0076-0.0030j  -0.0068j  -0.0099  0.0011+0.0047j  0.0031+0.0019j  -0.0034-0.0076j ]
[ -0.0061-0.0090j  -0.0006-0.0091j  0.0007j  0.0056+0.0033j  0.0011-0.0047j  -0.0021  0.0011+0.0069j  -0.0001-0.0074j ]
[ -0.0096+0.0057j  -0.0040j  0.0009-0.0074j  0.0001-0.0028j  0.0031-0.0019j  0.0011-0.0069j  -0.0014  -0.0071+0.0047j ]
[ 0.5000-0.0065j  0.0026+0.0005